In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('/Users/husky0607/Github/mlops-zoomcamp/week1/yellow_tripdata_2022-01.parquet')

In [3]:
len(df.columns)

19

In [4]:
def process_df(df):
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds()/60
    return df

In [6]:
df = df.pipe(process_df)

In [7]:
np.std(df['duration'])

46.445295712725304

In [8]:
df_filter = df.loc[(df['duration'] <= 60) & (df['duration'] >= 1)]

In [9]:
len(df_filter) / len(df)

0.9827547930522406

In [10]:
df_filter[['PULocationID',	'DOLocationID']].head()

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163


In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
df.shape

(2463931, 20)

In [13]:
enc = OneHotEncoder(handle_unknown='ignore')
df_dict = df_filter[['PULocationID','DOLocationID']].to_dict(orient='records')
# X = enc.fit_transform(df[['PULocationID',	'DOLocationID']])

In [14]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
X = v.fit_transform(df_dict)

In [15]:
X_trans = enc.fit_transform(X)

In [16]:
X_trans.shape

(2421440, 515)

In [17]:
y = df_filter['duration']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_trans, y)

In [20]:
from sklearn.metrics import mean_squared_error
y_pred = reg.predict(X_trans)
mean_squared_error(y, y_pred, squared=False)

6.986190725238826

In [38]:
test_df = pd.read_parquet('/Users/husky0607/Github/mlops-zoomcamp/week1/yellow_tripdata_2022-02.parquet')

In [39]:
test_df = test_df.pipe(process_df)

In [44]:
test_df = test_df.loc[(test_df['duration'] >= 1) & (test_df['duration'] <= 60)]

In [45]:
X_test_dict = v.transform(test_df[['PULocationID','DOLocationID']].to_dict(orient='records'))

In [46]:
X_test_transform = enc.transform(X_test_dict)

In [47]:
y_pred_test = reg.predict(X_test_transform)
y_test = test_df['duration']
mean_squared_error(y_test, y_pred_test, squared=False)

7.786407655353051